In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# from inference import SegmentationModel
# segm_model = SegmentationModel('unet.pth')
# pred_mask = segm_model.predict_mask('data/tasks/bac770ea-c9b2-4199-8f5d-245b9cc1e215')

In [3]:
from pathlib import Path
import torch
from torch.utils.data import Dataset
from albumentations.pytorch.transforms import img_to_tensor
from albumentations import (
    PadIfNeeded,
    Compose,
    Normalize,
)
import cv2
import numpy as np

In [4]:
import torch
from torch.utils.data import DataLoader
from dataset import slice_images_masks, slice_image
from pathlib import Path
from model import UNet11

In [5]:
device = 'cpu'
full_size = 4096
n_splits = 8

In [6]:
image_dir = 'data/tasks/bac770ea-c9b2-4199-8f5d-245b9cc1e215'
images, masks, source_image_padding = slice_images_masks([image_dir], full_size=full_size, n_splits=n_splits)

data/tasks/bac770ea-c9b2-4199-8f5d-245b9cc1e215


In [7]:
source_image_padding

{'data/tasks/bac770ea-c9b2-4199-8f5d-245b9cc1e215': [398, 349]}

In [8]:
default_transform = Compose([
    Normalize(p=1),
], p=1)

In [9]:
image_tensors = [img_to_tensor(default_transform(image=img)['image']) for img in images]

In [10]:
model = UNet11(pretrained=False)
state = torch.load(open('unet.pth', 'rb'), map_location=device)
model.load_state_dict(state)
model = model.to(device)

In [15]:
%%time

pred_outputs = []
with torch.no_grad():
    model.eval()
    for inputs in image_tensors:
        inputs = torch.unsqueeze(inputs, dim=0)
        inputs = inputs.to(device)
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs) > 0.3
        pred_outputs.append(outputs)
pred_outputs = torch.squeeze(torch.cat(pred_outputs))
pred_outputs = pred_outputs.detach().cpu().numpy()

CPU times: user 20min 39s, sys: 2min 4s, total: 22min 44s
Wall time: 7min 45s


In [24]:
pred_outputs.shape

(64, 512, 512)

In [86]:
pred_mask = combine_tiles(pred_outputs, full_size, n_splits)
row_pad, col_pad = source_image_padding[image_dir]
pred_mask = remove_padding(pred_mask, full_size, row_pad, col_pad)

In [87]:
# pred_mask_3ch = np.stack([pred_mask]*3, axis=2)

In [88]:
pred_mask.shape

(3698, 3747)

In [89]:
pred_mask = (pred_mask * 255).astype(np.uint8)
image = Image.fromarray(pred_mask, mode='L')
image.save(Path(image_dir) / 'mask.png')

In [50]:
from PIL import Image

In [54]:
img = Image.open('data/raw/n1_SELECTED/source.png')

In [55]:
a = np.array(img)

In [56]:
Image.fromarray(a[:512,:512], mode='L').save('data/raw/source_small.png')

In [99]:
import requests

def count_words_at_url(url):
    resp = requests.get(url)
    return len(resp.text.split())

In [11]:
# from rq import Queue
# from redis import Redis
# # from somewhere import count_words_at_url

# # Tell RQ what Redis connection to use
# redis_conn = Redis()
# q = Queue(connection=redis_conn)  # no args implies the default queue

# # Delay execution of count_words_at_url('http://nvie.com')
# job = q.enqueue(count_words_at_url, 'http://nvie.com')
# print(job.result)   # => None

# # Now, wait a while, until the worker is finished
# time.sleep(2)
# print(job.result)   # => 889

In [47]:
import redis
client = redis.Redis()

In [48]:
client.lpush('task-queue', 'd9c80eb7-f0f3-4663-9c31-e3ff5261e939')

1

In [42]:
r = client.lpop('task-queue')
(r,)

(b'd9c80eb7-f0f3-4663-9c31-e3ff5261e939',)

In [46]:
r.decode('utf-8')

'd9c80eb7-f0f3-4663-9c31-e3ff5261e939'